## play by play data

###  import modules

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy

### import data 

In [2]:
dm = pd.read_csv('out_data/reshape.csv')
dm = dm.drop('Unnamed: 0', axis=1)

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dm.columns

Index(['AdvantageType', 'Assist1Player', 'Assist2Player', 'DrawnByPlayerName',
       'DrawnByPlayerNumber', 'DrawnByTeamCode', 'EventNumber', 'EventType',
       'GameDate', 'GameNumber', 'GoalNumber', 'GoalTime', 'HTeamCode',
       'HitteePlayerName', 'HitteePlayerNumber', 'HitteeTeamCode', 'Period',
       'PlayerName', 'PlayerNumber', 'Season', 'ShotPlayerName',
       'ShotPlayerNumber', 'ShotResult', 'ShotTeamCode', 'ShotType',
       'TeamCode', 'VTeamCode', 'Zone', 'endtime', 'starttime', 'VPlayer',
       'HPlayer', 'VPosition', 'HPosition'],
      dtype='object')

- sort values of data frame and organize the order of the columns

In [18]:
dm = dm.sort_values(['Season', 'GameNumber', 'Period', 'EventNumber'], ascending=[True, True, True, True])
dm = dm[['Season', 'GameNumber', 'Period', 'AdvantageType', 'EventNumber', 'EventType', 'Zone', 'VTeamCode', 'VPlayer', 'VPosition', 'HTeamCode', 'HPlayer', 'HPosition', 'TeamCode', 'PlayerNumber', 'PlayerName', 'PlayerPosition', 'starttime', 'endtime', 'GoalNumber', 'GoalTime', 'Assist1Player', 'Assist2Player', 'ShotType', 'ShotResult', 'ShotTeamCode', 'ShotPlayerNumber', 'ShotPlayerName', 'HitteeTeamCode', 'HitteePlayerNumber', 'HitteePlayerName', 'DrawnByTeamCode', 'DrawnByPlayerNumber', 'DrawnByPlayerName']]

- assign value to home and away team based on the on-ice event. Use forward and backwrad fill to fill in home team code and away team code per game

In [19]:
dm['HTeamCode'] = np.where(dm['PlayerNumber'] == dm['HPlayer'], dm['TeamCode'], np.nan)

In [20]:
dm['HTeamCode'] = dm.groupby('GameNumber')['HTeamCode'].ffill()
dm['HTeamCode'] = dm.groupby('GameNumber')['HTeamCode'].bfill()

In [21]:
dm['VTeamCode'] = np.where(dm['PlayerNumber'] == dm['VPlayer'], dm['TeamCode'], np.nan)

In [22]:
dm['VTeamCode'] = dm.groupby('GameNumber')['VTeamCode'].ffill()
dm['VTeamCode'] = dm.groupby('GameNumber')['VTeamCode'].bfill()

- each period starts off with a faceoff in the neutral zone. Fill in 'NaN' with 'N' (neutral).

In [23]:
dm['Zone'] = dm['Zone'].fillna('N')

In [24]:
dm['TeamCode'].fillna(dm.groupby('GameNumber')['VTeamCode'].ffill(), inplace=True)

- use **numpy** to fill in player position and player number depending if the player involved in the on-ice event played for the home or away team.

In [25]:
dm['PlayerPosition'] = np.where((dm['TeamCode'] == dm['VTeamCode']) & (dm['PlayerNumber'] == dm['VPlayer']), dm['VPosition'], dm['HPosition'])

In [26]:
dm['PlayerNumber'] = np.where((dm['TeamCode'] == dm['VTeamCode']) & (dm['PlayerPosition'] == dm['VPosition']), dm['VPlayer'], dm['HPlayer'])

- Assign a value of 0 to both variables **'starttime'** and **'endtime'**, since the first faceoff of each period has no time duration. 

In [27]:
dm['starttime'].fillna(0, inplace=True)
dm['endtime'].fillna(0, inplace=True)

In [28]:
dm.to_csv('out_data/play_by_play.csv', index='False', sep=',')

#### player quality follows